In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

import textwrap

c:\Users\Mittal\anaconda3\envs\langchain_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""   # open-AI api key

In [3]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = FAISS.load_local(
    "../vector_store/faiss_store",
    embeddings,
    allow_dangerous_deserialization=True
)

In [4]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

In [ ]:
def get_overview_text(k=12):
    """
    Pulls a broad set of chunks from your notes to show the LLM
    what the document roughly contains.
    """
    docs = vectorstore.similarity_search("overview of this document", k=k)
    combined = "\n\n".join(d.page_content for d in docs)
    return combined

In [7]:
def generate_chapter_outline(max_sections=6):
    """
    Asks the LLM to propose a chapter/section breakdown
    based on overview text.
    """
    overview_text = get_overview_text(k=15)

    prompt = f"""
    You are an AI tutor. Based on the content below, propose a clean,
    ordered list of up to {max_sections} chapters/sections that would
    make sense for organizing this material.

    For each chapter, give:
    - A short title
    - A 1–2 line description

    Return in this format exactly:

    1. <Title> - <short description>
    2. <Title> - <short description>
    ...

    Content:
    {overview_text}
    """

    resp = llm.invoke(prompt)
    return resp.content

chapter_outline = generate_chapter_outline()
print(chapter_outline)


1. Introduction to Software Engineering - An overview of software engineering, its importance, and the key concepts involved in software production from conception to maintenance.

2. Software Development Lifecycle - A detailed exploration of the stages of software development, including requirement analysis, design, coding, testing, and maintenance.

3. System Design and Architecture - Discussion of high-level and low-level design processes, focusing on the architectural specifications and module design needed for effective system functionality.

4. Implementation and Coding - An explanation of the coding phase where developers write the actual code for system modules, ensuring adherence to professional standards and integration needs.

5. Testing Strategies in Software Development - An overview of different testing phases, including system testing and user acceptance testing, to ensure the software meets functional and non-functional requirements.

6. Ethical Principles in Software E

In [ ]:
def summarize_chapter(chapter_title, k=12):
    docs = vectorstore.similarity_search(chapter_title, k=k)

    # If retrieval was weak → expand automatically
    if len(docs) < 3:
        docs = vectorstore.similarity_search(chapter_title.split()[0], k=k+5)

    combined_text = "\n\n".join(d.page_content for d in docs)

    prompt = f"""
    You are an AI study assistant.
    Create a chapter summary for: **{chapter_title}**

    Using ONLY the content below, produce:

    6-10 bullet point summary
    5 important concepts
    Try to use full sentences
    Don't skip if the content is small (less available)

    Content:
    {combined_text}

    Final Summary:
    """

    resp = llm.invoke(prompt)
    return resp.content if resp else "⚠ No summary generated"


In [12]:
def generate_chapter_flashcards(chapter_title, k=10, num_cards=10):
    docs = vectorstore.similarity_search(chapter_title, k=k)

    if len(docs) < 3:
        docs = vectorstore.similarity_search(chapter_title.split()[0], k=k+5)

    combined_text = "\n\n".join(d.page_content for d in docs)

    prompt = f"""
    Generate **{num_cards} flashcards** for the topic: {chapter_title}

    Format only as:

    Q: <question>
    A: <answer>

    Content:
    {combined_text}

    Flashcards:
    """

    resp = llm.invoke(prompt)
    return resp.content


In [16]:
chapter_titles = [
    "Introduction to Software Engineering",
]

chapter_data = {}
for title in chapter_titles:
    print(f"\n{title}")

    summary = summarize_chapter(title)
    flash = generate_chapter_flashcards(title)

    print(summary)
    print("\nFLASHCARDS:\n", flash)
    print("\n"+"="*80)




Introduction to Software Engineering
### Chapter Summary for "Introduction to Software Engineering"

- Software systems are abstract and intangible, and software engineering is crucial for the operation of government, society, and various institutions.
- Software is defined as computer programs and their associated documentation, which can be developed for specific customers or for a general market.
- Software engineering is an engineering discipline that encompasses all aspects of software production, from initial conception to operation and maintenance.
- The fundamental activities of software engineering include software specification, development, validation, and evolution.
- Key challenges in software engineering include handling increasing diversity, meeting demands for quicker delivery, and ensuring the development of trustworthy software.
- The costs of software engineering are predominantly made up of development costs (about 60%) and testing costs (around 40%), with evolutio

In [15]:
chapter_titles = [
    "Software Development Lifecycle",
]

chapter_data = {}
for title in chapter_titles:
    print(f"\n{title}\n")

    summary = summarize_chapter(title)
    flash = generate_chapter_flashcards(title)

    print(summary)
    print("\nFLASHCARDS:\n", flash)
    print("\n"+"="*80)


Software Development Lifecycle

### Chapter Summary: Software Development Lifecycle

- The Software Development Lifecycle (SDLC) is a structured process for designing, developing, and testing high-quality software.
- SDLC serves as a detailed methodology that outlines each step involved in software development, facilitating a systematic approach.
- The primary goal of SDLC is to deliver software that meets user requirements while being high-quality and maintainable.
- Selecting the appropriate SDLC model is crucial for project success and depends on various factors such as project requirements, budget, and timeline.
- Different SDLC models, including Waterfall, Agile, V-Model, Spiral, and Incremental, offer distinct approaches tailored to specific project needs.
- The Spiral Model is notable for its iterative and incremental approach, allowing for flexibility and emphasizing risk management throughout the development process.
- The V-Model integrates testing and validation with each d